<a href="https://colab.research.google.com/github/Rougephoenix/AI_stuff/blob/main/Residual_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.utils.data import dataset
epochs = 25
batch_size = 100
lr = 0.001

transform = transforms.Compose([transforms.Pad(4),
                                transforms.RandomHorizontalFlip(),
                                transforms.CenterCrop(32),
                                transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root = "./root",train = True,transform = transform,download = True)
test_dataset = torchvision.datasets.CIFAR10(root = './root',train = False,transform = transforms.ToTensor(),download = False)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,shuffle = True,batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,shuffle = False,batch_size = batch_size)

100%|██████████| 170498071/170498071 [00:13<00:00, 12650564.57it/s]


Extracting ./root/cifar-10-python.tar.gz to ./root


In [ ]:
device

device(type='cuda')

In [ ]:
def conv3x3(in_channels,out_channels,stride = 1):
  return nn.Conv2d(in_channels,out_channels,kernel_size = 3,stride = stride,bias = False,padding=1)

class ResidualBlock(nn.Module):
  def __init__(self,in_channels,out_channels,stride = 1,downsample = None):
    super(ResidualBlock,self).__init__()
    self.conv1 = conv3x3(in_channels,out_channels,stride)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU(inplace=True)
    self.conv2 = conv3x3(out_channels,out_channels)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.downsample = downsample

  def forward(self,x):
    residual = x
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    if self.downsample:
      residual = self.downsample(x)
    out = out + residual
    out = self.relu(out)
    return out


In [ ]:
class ResNet(nn.Module):
  def __init__(self,block,layers,num_classes=10):
    super(ResNet,self).__init__()
    self.in_channels = 16
    self.conv1 = conv3x3(3,16)
    self.bn = nn.BatchNorm2d(16)
    self.relu = nn.ReLU(inplace = True)
    self.layer1 = self.make_layer(block,16,layers[0])
    self.layer2 = self.make_layer(block,32,layers[1],2)
    self.layer3 = self.make_layer(block,64,layers[2],2)
    self.avg_pool = nn.AvgPool2d(8)
    self.fc = nn.Linear(64,num_classes)


  def make_layer(self,block, out_channels,blocks,stride = 1):
    downsample = None
    if (stride != 1) or (self.in_channels != out_channels):
      downsample = nn.Sequential(conv3x3(self.in_channels,out_channels,stride = stride),
                                 nn.BatchNorm2d(out_channels))
    layers = []
    layers.append(block(self.in_channels,out_channels,stride,downsample))
    self.in_channels = out_channels
    for i in range(1,blocks):
      layers.append(block(out_channels,out_channels))
    return nn.Sequential(*layers)


  def forward(self,x):
    out = self.conv1(x)
    out = self.bn(out)
    out = self.relu(out)
    out = self.layer1.forward(out)
    out = self.layer2.forward(out)
    out = self.layer3.forward(out)
    out = self.avg_pool(out)
    out = out.view(-1,64) #!!!!!!!!!!!!!!!!!!!!!!!!
    out = self.fc(out)
    return out


In [ ]:
model = ResNet(ResidualBlock,[2,2,2]).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(),lr = lr)

In [ ]:
decay = 0
model.train

for epoch in range(epochs):

  if (epoch+1)%20 == 0:
    decay +=1
    optimiser.param_groups[0]['lr'] = lr*(0.05**decay)

  for i,(images,labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = criterion(outputs,labels)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if (i+1) % 100 == 0:
      print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}".format(epoch+1,epochs,i+1,len(train_loader),loss.item()))

Epoch [1/25], Step [100/500], Loss: 1.6527
Epoch [1/25], Step [200/500], Loss: 1.2913
Epoch [1/25], Step [300/500], Loss: 1.4041
Epoch [1/25], Step [400/500], Loss: 1.0468
Epoch [1/25], Step [500/500], Loss: 1.1140
Epoch [2/25], Step [100/500], Loss: 1.0219
Epoch [2/25], Step [200/500], Loss: 0.8396
Epoch [2/25], Step [300/500], Loss: 0.9808
Epoch [2/25], Step [400/500], Loss: 0.8196
Epoch [2/25], Step [500/500], Loss: 0.8215
Epoch [3/25], Step [100/500], Loss: 0.9307
Epoch [3/25], Step [200/500], Loss: 0.7481
Epoch [3/25], Step [300/500], Loss: 0.8097
Epoch [3/25], Step [400/500], Loss: 0.5884
Epoch [3/25], Step [500/500], Loss: 0.7067
Epoch [4/25], Step [100/500], Loss: 0.5434
Epoch [4/25], Step [200/500], Loss: 0.5775
Epoch [4/25], Step [300/500], Loss: 0.5328
Epoch [4/25], Step [400/500], Loss: 0.5852
Epoch [4/25], Step [500/500], Loss: 0.7512
Epoch [5/25], Step [100/500], Loss: 0.4919
Epoch [5/25], Step [200/500], Loss: 0.7117
Epoch [5/25], Step [300/500], Loss: 0.6510
Epoch [5/25

In [ ]:
#Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 86.56 %


In [ ]:
model.eval()

with torch.no_grad():
  correct = 0
  total = 0
  for images,labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  accuracy = correct/total * 100
  print(accuracy)

86.56
